In [113]:
!pip freeze
!pip3 install numpy
!pip3 install pandas
!pip3 install sklearn
!pip3 install matplotlib
!pip3 install pydotplus
!pip3 install six
!pip install -Uqq ipdb

absl-py==1.2.0
aeppl==0.0.33
aesara==2.7.9
aiohttp==3.8.1
aiosignal==1.2.0
alabaster==0.7.12
albumentations==1.2.1
altair==4.2.0
appdirs==1.4.4
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
async-timeout==4.0.2
asynctest==0.13.0
atari-py==0.2.9
atomicwrites==1.4.1
attrs==22.1.0
audioread==3.0.0
autograd==1.4
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.1
blis==0.7.8
bokeh==2.3.3
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==2.0.8
certifi==2022.6.15
cffi==1.15.1
cftime==1.6.1
chardet==3.0.4
charset-normalizer==2.1.1
click==7.1.2
clikit==0.6.2
cloudpickle==1.5.0
cmake==3.22.6
cmdstanpy==1.0.7
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
cons==0.4.5
contextlib2==0.5.5
convertdate==2.4.0
crashtest==0.3.1
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.3.0
cvxpy==1.2.1
cycler==0.11.0
cymem==2.0.6
Cython==0.29.32
daft==0.0.4
dask==2022.2.0
datascience==0.17.5
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.

Problem 2: Decisions, decisions

Implement ID3 from scratch

Below is the prep and helper functions.

In [114]:
import pandas as pd
import numpy as np
import ipdb
# help from https://zohaib.me/debugging-in-google-collab-notebook/

In [115]:
# preparing data for use
url = 'https://raw.githubusercontent.com/f1nn3g4n/SENG474Test/ad161e4c364f92adc9730fe6c06edd2769b9f355/elections_clean.csv'
df = pd.read_csv(url, sep='\t')
# help from https://datascienceparichay.com/article/randomly-shuffle-pandas-dataframe-rows/#:~:text=There%20are%20a%20number%20of%20ways%20to%20shuffle,sklearn.utils%20to%20shuffle%20your%20dataframe.%20Here%E2%80%99s%20the%20syntax%3A
df = df.sample(frac=1)

total = df.shape[0]
train_size = int(round(total*0.7))

df_train = df.iloc[:train_size,:]
df_valid = df.iloc[train_size:,:]

In [116]:
category_features = ['ethnic_male', 'ethnic_female', 'state', 'county',
                    'education',]
label_vector = 'democrat'
values = df_train[label_vector].unique()
# dict to be populated with counts of feature repeats
feature_repeats = {}

In [117]:
# guidance from https://medium.com/geekculture/step-by-step-decision-tree-id3-algorithm-from-scratch-in-python-no-fancy-library-4822bbfdd88f

In [128]:
def demOrRep(value):
  ''' Simple string return for tree leaves on if democrat or republican '''
  # if type(value) == int:
  return 'democrat' if value else 'republican'
  # else:
  #   return value

In [119]:
def findInfoGain(feature, train_data):
  ''' Takes a feature and training data and finds information gain, 
  for feature splitting. 
  '''
  feature_vals = feature.unique()
  n = train_data.shape[0]
  info_gain = 0
  feature_val_counts = feature.value_counts(sort=False)
  tree_entropy = 0
  tree_ent_flag = False
  i = 0
  
  # feature entropy
  for feat_val, count in feature_val_counts.iteritems():
    i+=1
    f_data = train_data[train_data[feature.name]==feat_val]
    entropy = 0
    for value in values:
      if not tree_ent_flag:
        # two time tree entropy calculation
        tree_ent_flag = tree_entropy != 0
        t_val_count = train_data[train_data[label_vector]==value].shape[0]
        t_prob = t_val_count/n
        tree_entropy += -t_prob*np.log2(t_prob)

      val_count = f_data[f_data[label_vector]==value].shape[0]
      if val_count != 0:
          prob = val_count/count
          entropy += -prob*np.log2(prob)
    info_gain += count/n * entropy

  return tree_entropy - info_gain

In [120]:
def findSplitFeature(train_data):
  max_ig = -1
  max_info_feature = None

  for category in category_features:
    category_ig = findInfoGain(train_data[category], train_data)
    if max_ig < category_ig:
      max_ig = category_ig
      max_info_feature = train_data[category]

  return max_info_feature

Below is the tree building function and resulting tree.

In [121]:
class TreeNode:
  def __init__(self, data):
    self.name = data.name # where data is a pandas Series
    self.children = {} # dictionary of str:node/str pairs
    self.gini = 0
    self.prob = 0
    for value in data.unique():
      self.children[value] = None

  def __str__(self):
    print('Node: '+self.name)

  def findMaxDepth(self, d):
    max_d = d+1
    for child in self.children:
      if type(self.children[child]) is not str:
        child_d = self.children[child].findMaxDepth(d+1)
        if max_d < child_d:
          max_d = child_d
    return max_d

  def calcGiniImpurity(self):
    total = 0
    


In [122]:
def printTree(node, tab_count):
  ''' Creates a visualization of the ID3 tree '''
  tabs = tab_count*'  '
  print(tabs+node.name)
  tabs += '  '
  for child in node.children:
    print(tabs+str(child))
    if type(node.children[child]) is str:
      print(tabs+'  '+node.children[child])
    else:
      printTree(node.children[child],tab_count+2)

In [123]:
def assignFeatRe(feature):
  ''' Increases count of feature repeat '''
  if feature_repeats.get(feature.name) != None:
    feature_repeats[feature.name]+=1
  else:
    feature_repeats[feature.name]=1

In [124]:
def buildID3Tree(train_data, cur_node, root):
  ''' Takes in training data, returns a tree using the TreeNode class above. '''
  if root == None:
    root_feature = findSplitFeature(train_data)
    root = TreeNode(root_feature)
    cur_node = root
    assignFeatRe(root_feature)

  for child in cur_node.children:
    child_data = train_data[train_data[cur_node.name]==child]
    child_count = child_data.shape[0]

    for value in values:
      val_count = child_data[child_data[label_vector] == value].shape[0]

      if val_count == child_count:
        # reached a leaf
        cur_node.children[child] = demOrRep(value)
        train_data = train_data[train_data[cur_node.name]!=child]
      elif val_count != 0:
        # need to add node
        child_feature = findSplitFeature(child_data)
        child_node = TreeNode(child_feature)
        # track repeat features
        assignFeatRe(child_feature)
        # assign prob of node, for gini calc
        child_node.prob = val_count/child_count
        cur_node.children[child] = child_node
        buildID3Tree(child_data, child_node, root)

  cur_node.calcGiniImpurity()
  return root

In [129]:
df_train_copy = df_train.copy()
tree = buildID3Tree(df_train_copy,None,None)

In [131]:
printTree(tree,0)

county
  Lubbock County
    republican
  Chisago County
    republican
  Walton County
    republican
  Tillman County
    republican
  Pawnee County
    republican
  Ionia County
    republican
  Platte County
    republican
  Androscoggin County
    republican
  Randolph County
    ethnic_male
      WHITE MALE
        republican
      BLACK MALE
        democrat
  Lincoln County
    state
      TN
        republican
      OK
        republican
      NE
        republican
      NM
        republican
      KY
        republican
      WA
        republican
      SD
        republican
      MN
        republican
      ID
        republican
      NV
        republican
      OR
        democrat
      MO
        republican
      MS
        republican
      WI
        republican
      ME
        democrat
      WV
        republican
      WY
        republican
      AR
        republican
  Douglas County
    state
      NV
        republican
      GA
        democrat
      MN
        republic

In [ ]:
# df_class = pd.read_csv('https://raw.githubusercontent.com/f1nn3g4n/SENG474Test/main/class2.csv')
# df_class = df_class.drop('Unnamed: 4', axis=1)
# category_features = ['Outlook', 'Temperature', 'Humidity']
# label_vector = "Play?"
# values = ['Yes', 'No']
# tree2 = buildID3Tree(df_class, None, None)
# printTree(tree2,0)

Part 2: Gini coefficient

In [ ]:
def findInfoGainGini(feature, train_data):
  ''' Takes a feature and training data and finds information gain, 
  for feature splitting. 
  '''
  feature_vals = feature.unique()
  n = train_data.shape[0]
  info_gain = 0
  feature_val_counts = feature.value_counts(sort=False)
  tree_entropy = 0
  tree_ent_flag = False
  i = 0
  
  # feature entropy
  for feat_val, count in feature_val_counts.iteritems():
    i+=1
    f_data = train_data[train_data[feature.name]==feat_val]
    entropy = 0
    for value in values:
      if not tree_ent_flag:
        # two time tree entropy calculation
        tree_ent_flag = tree_entropy != 0
        t_val_count = train_data[train_data[label_vector]==value].shape[0]
        t_prob = t_val_count/n
        tree_entropy += -t_prob*np.log2(t_prob)

      val_count = f_data[f_data[label_vector]==value].shape[0]
      if val_count != 0:
          prob = val_count/count
          entropy += -prob*np.log2(prob)
    info_gain += count/n * entropy

  return tree_entropy - info_gain

# Analysis

In [132]:
# some quick checks
print(feature_repeats)
print()
print(tree.findMaxDepth(0))

{'county': 2, 'ethnic_male': 64, 'state': 316, 'ethnic_female': 8}

2


In [133]:
def calcError(dataset, tree):
  ''' Calculates and returns the error of a tree from a dataset '''
  total = dataset.shape[0]
  total_right = 0

  for index, row in dataset.iterrows():
    cur_node = tree
    in_tree = True
    elected = demOrRep(row['democrat'])
    while in_tree:
      row_node_value = row[cur_node.name]
      child = cur_node.children.get(row_node_value)

      if child == None:
        in_tree = False
      elif type(child) == str:
        in_tree = False
        if child == elected:
          total_right+=1
      else:
        cur_node = child
  
  return 1 - total_right/total

In [134]:
train_error = calcError(df_train, tree)
valid_error = calcError(df_valid, tree)

In [135]:
print('training error', train_error)
print('valid_error', valid_error)

training error 0.0
valid_error 0.6878980891719746
